# Readme

主要目的是测试使用本征态作为初态进行学习的效果，对比使用scar态进行层析

## 1.生成本征态

这里选择xorX模型（存在scar态）

- 给定模型参数
- 计算本征态

In [12]:
import torch as tc

import torch

def pauli_string(n, positions, paulis, sigma_x, sigma_z, identity):
    matrices = []
    for pos in range(1, n + 1):
        if pos in positions:
            idx = positions.index(pos)
            p = paulis[idx]
            if p == 'x':
                mat = sigma_x.clone()
            elif p == 'z':
                mat = sigma_z.clone()
            else:
                raise ValueError(f"Invalid Pauli type: {p}")
        else:
            mat = identity.clone()
        matrices.append(mat)
    if not matrices:
        return torch.eye(1, dtype=torch.complex64)
    term = matrices[0]
    for mat in matrices[1:]:
        term = torch.kron(term, mat)
    return term

def construct_xorX(n, lamda, delta, J):
    sigma_x = torch.tensor([[0, 1], [1, 0]], dtype=torch.complex64)
    sigma_z = torch.tensor([[1, 0], [0, -1]], dtype=torch.complex64)
    identity = torch.eye(2, dtype=torch.complex64)
    
    H = torch.zeros((2**n, 2**n), dtype=torch.complex64)
    
    # 第一部分：λ项（i从2到n-1）
    if n >= 3:
        for i in range(2, n):  # i的取值为2 ≤ i ≤ n-1
            term1 = pauli_string(n, [i], ['x'], sigma_x, sigma_z, identity)
            term2 = pauli_string(n, [i-1, i, i+1], ['z', 'x', 'z'], sigma_x, sigma_z, identity)
            H += lamda * (term1 - term2)
    
    # 第二部分：Δ项（i从1到n）
    for i in range(1, n + 1):
        term = pauli_string(n, [i], ['z'], sigma_x, sigma_z, identity)
        H += delta * term
    
    # 第三部分：J项（i从1到n-1）
    for i in range(1, n):
        term = pauli_string(n, [i, i+1], ['z', 'z'], sigma_x, sigma_z, identity)
        H += J * term
    
    return H

# 示例用法
if __name__ == "__main__":
    n = 5       # 量子比特数
    lamda = 1.0 # λ参数
    delta = 0.5 # Δ参数
    J = -0.2    # J参数
    
    H = construct_xorX(n, lamda, delta, J)
    print("哈密顿量矩阵维度:", H.shape)
    _, vec = tc.linalg.eig(H)
    print(vec)

# param = {}
# H_xorX = construct_xorX()
# eig_states, _ = tc.eig(H_xorX)

哈密顿量矩阵维度: torch.Size([32, 32])
tensor([[ 0.0000e+00-0.j,  0.0000e+00+0.j,  0.0000e+00+0.j,  ...,
          0.0000e+00+0.j,  0.0000e+00+0.j,  0.0000e+00+0.j],
        [ 3.4727e-09+0.j,  1.3141e-07+0.j,  1.1330e-05+0.j,  ...,
          0.0000e+00+0.j,  0.0000e+00+0.j,  0.0000e+00+0.j],
        [-1.1738e-08-0.j,  2.6325e-01+0.j,  1.7976e-01+0.j,  ...,
          0.0000e+00+0.j,  0.0000e+00+0.j,  0.0000e+00+0.j],
        ...,
        [ 2.0806e-18+0.j,  3.4156e-10+0.j,  1.1487e-07+0.j,  ...,
          0.0000e+00+0.j,  0.0000e+00+0.j,  0.0000e+00+0.j],
        [-6.2368e-01-0.j,  1.1312e-07+0.j, -2.2582e-07+0.j,  ...,
          0.0000e+00+0.j,  0.0000e+00+0.j,  0.0000e+00+0.j],
        [ 0.0000e+00-0.j,  0.0000e+00+0.j,  0.0000e+00+0.j,  ...,
          0.0000e+00+0.j,  0.0000e+00+0.j,  1.0000e+00+0.j]])


## 2.验证生成的本征态

- 对生成的本征态进行演化
- 计算初态和末态的内积的模方（等于1表示二者只差一个相位）

In [15]:
from WorkFlow import TimeEvol

length = 5
lamda = 1.0
delta = 0.1
J = 1

H_xorX = construct_xorX(n=length, lamda=lamda, delta=delta, J=J)
_, vecs = tc.linalg.eig(H_xorX)
states_init = vecs.T.reshape([2**length] + [2]*length)
print(states_init.shape)

evol_param = {
    'length': length,
    'lambda': lamda,
    'delta': delta,
    'J': J,
    'time_tot': 1
}
evol_states = TimeEvol.xorX_mul_states_evl(states=states_init, para=evol_param)
init = states_init.reshape((32, -1))
evol = evol_states.reshape((32, -1))
a = tc.einsum('ab, ab -> a', init, evol.conj())
print(tc.einsum('i, i->i', a, a.conj()))

torch.Size([32, 2, 2, 2, 2, 2])
tensor([0.9998+0.j, 0.9998+0.j, 0.9997+0.j, 1.0000+0.j, 0.9997+0.j, 1.0000+0.j, 0.9995+0.j,
        1.0000+0.j, 0.9997+0.j, 0.9995+0.j, 0.9997+0.j, 0.9998+0.j, 0.9998+0.j, 0.9998+0.j,
        0.9991+0.j, 1.0000+0.j, 0.9995+0.j, 0.9995+0.j, 0.9998+0.j, 0.9991+0.j, 0.9997+0.j,
        0.9997+0.j, 0.9997+0.j, 0.9997+0.j, 0.9997+0.j, 0.9997+0.j, 0.9997+0.j, 0.9997+0.j,
        1.0000+0.j, 1.0000+0.j, 1.0000+0.j, 1.0000+0.j])


## 3.使用本征态作为初态和scar态作为初态进行学习

需要调用gpu，通过脚本提交任务运行

In [1]:
# 使用本征态作为初态
import sys

# 忽略 Jupyter Notebook 的参数
sys.argv = sys.argv[:1]

from WorkFlow.main import *
import argparse



if __name__ == "__main__":

    # 创建一个新的解析器
    parser = argparse.ArgumentParser(description='Combined Argument Parser')

    # 创建物理模型参数组
    model_group = parser.add_argument_group('Physical Model Parameters')
    model_group.add_argument('--J', type=float, default=1)
    model_group.add_argument('--delta', type=float, default=0.1)
    model_group.add_argument('--lambda_', type=float, default=1)

    # 创建训练参数组
    train_group = parser.add_argument_group('Training Parameters')
    train_group.add_argument('--sample_num', type=int, default=10, help='Number of samples')
    train_group.add_argument('--length', type=int, default=10, help='Length of the input')
    train_group.add_argument('--data_type', type=str, default='eig', help='Type of data')
    train_group.add_argument('--loss', type=str, default='multi_mags', help='Loss function type')
    train_group.add_argument('--time_interval', type=float, default=0.2, help='Time interval for training')
    train_group.add_argument('--evol_num', type=int, default=1, help='Number of evolutions')
    train_group.add_argument('--tau', type=float, default=0.02, help='Time constant')

    # 解析参数
    args = parser.parse_args()

    # 将参数分组
    model_params = {
        'J': args.J,
        'delta': args.delta,
        'lambda': args.lambda_,
    }

    train_params = {
        'sample_num': args.sample_num,
        'length': args.length,
        'data_type': args.data_type,
        'loss': args.loss,
        'time_interval': args.time_interval,
        'evol_num': args.evol_num,
        'tau': args.tau
    }

    csv_file_path = '/data/home/scv7454/run/GraduationProject/Data/xorX_(J={})(delta={})(lambda={}).csv'.format(model_params['J'], model_params['delta'], model_params['lambda'])

    m = 0
    tot_gate_fidelity = 0
    tot_spectrum_diff = 0
    tot_similarity = 0
    tot_H_diff = 0
    for i in range(2):
        m += 1
        init_train_para, init_test_para, return_model_para, evol_para, nn_para, save_para = pack_params(model_name='xorX', model_para=model_params, device=tc.device('cuda:0'), csv_file_path=csv_file_path, **train_params)
        gate_fidelity, spectrum_diff, similarity, H_diff = run_with_param(init_train_para, init_test_para, return_model_para, evol_para, nn_para, save_para)
        print(gate_fidelity, spectrum_diff, similarity, H_diff)
        tot_gate_fidelity += gate_fidelity
        tot_spectrum_diff += spectrum_diff
        tot_similarity += similarity
        tot_H_diff += H_diff
    avg_gate_fidelity = tot_gate_fidelity / m
    avg_spectrum_diff = tot_spectrum_diff / m
    avg_similarity = tot_similarity / m
    avg_H_diff = tot_H_diff / m
    print('average_gate_fidelity={}, avgerage_spectrum_diff={}, average_similarity={}, avg_H_diff={}'.format(float(avg_gate_fidelity), float(avg_spectrum_diff), float(avg_similarity), float(H_diff)))


RuntimeError: No CUDA GPUs are available